In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from preprocessing import *

In [2]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('data/보유리스크율_일반')
BASE_YYMM = '201912'
BASE_YYYY = BASE_YYMM[:4]

In [73]:
# 데이터 불러오기
일반_원수_직전3년연간경과보험료 = pd.read_excel(FILE_PATH / f'일반_원수_직전3년연간경과보험료_{202012}.xlsx',
    dtype={'FY': str, 'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'OGL_ELP_PRM': float})
일반_출재_직전3년연간경과보험료 = pd.read_excel(FILE_PATH / f'일반_출재_직전3년연간경과보험료_{202012}.xlsx',
    dtype={'FY': str, 'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'RN_ELP_PRM': float})
일반_원수_직전3년연간손해액 = pd.read_excel(FILE_PATH / f'일반_원수_직전3년연간손해액_{202012}.xlsx',
    dtype={'FY': str, 'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'PDC_CD': str, 'OGL_LOSS': float})
일반_출재_직전3년연간손해액 = pd.read_excel(FILE_PATH / f'일반_출재_직전3년연간손해액_{202012}.xlsx',
    dtype={'FY': str, 'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'PDC_CD': str, 'RN_LOSS': float})
일반_특약보종별_직전1년경과보험료 = pd.read_excel(FILE_PATH / f'일반_특약보종별_직전1년경과보험료_{BASE_YYMM}.xlsx',
    dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'RRNR_TTY_CD': str, 'ELP_PRM': float, 'RN_ELP_PRM': float}) \
    .rename(columns={'ELP_PRM': 'OGL_ELP_PRM'}) # TODO: ELP_PRM -> OGL_ELP_PRM로 수정하기
일반_특약보종별_직전1년손해액 = pd.read_excel(FILE_PATH / f'일반_특약보종별_직전1년손해액_{BASE_YYMM}.xlsx',
    dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'RRNR_TTY_CD': str, 'OGL_LOSS': float, 'RN_LOSS': float})
일반_특약수수료 = pd.read_excel(FILE_PATH / f'일반_특약수수료_{202012}.xlsx', dtype={'RRNR_TTY_CD': str, 'TTY_YR': str})
일반_특약정보 = pd.read_excel(FILE_PATH / f'일반_특약정보_{BASE_YYMM}.xlsx', dtype={'RRNR_TTY_CD': str, 'TTY_YR': str})
일반_상품정보 = pd.read_excel(FILE_PATH / '일반_상품정보.xlsx', dtype={'PDC_CD': str, 'PDGR_CD': str})
산업_손해율_변동계수 = pd.read_excel(FILE_PATH / '산업_손해율_변동계수.xlsx')

In [71]:
pd.options.display.float_format = '{:,.3f}'.format

# 특약정보
## 근재보험특약, 기술보험특약, 재물보험특약, 배상책임보험특약, 해외PST
일반_특약정보_가공 = 일반_특약정보.copy()
일반_특약정보_가공['TTY_CD_GRP'] = None
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("(재물보험|패키지)"), 'TTY_CD_GRP'] = '재물보험특약'
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("기술보험"), 'TTY_CD_GRP'] = '기술보험특약'
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("근재보험"), 'TTY_CD_GRP'] = '근재보험특약'
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("배상책임보험"), 'TTY_CD_GRP'] = '배상책임보험특약'
일반_특약정보_가공.loc[lambda x: x.TTY_CD_NM.str.contains("해외PST"), 'TTY_CD_GRP'] = '해외PST'

# 특약수수료
일반_특약수수료_가공 = 일반_특약수수료.copy()
일반_특약수수료_가공 = 일반_특약수수료_가공.merge(일반_특약정보_가공, on=['RRNR_TTY_CD', 'TTY_YR'], how='left').query('~TTY_CD_GRP.isna()')
일반_특약수수료_가공 = 일반_특약수수료_가공[['TTY_CD_GRP', 'TTY_YR', 'CMSN_ADD_RT', 'CMSN_MULT_RT', 'BSE_LSRT', 'PVSN_CMSN_RT', 'LWT_CMSN_RT', 'TOP_CMSN_RT']].drop_duplicates()

# 직전3년평균손해율
일반_원수_직전3년연간경과보험료_가공 = 일반_원수_직전3년연간경과보험료.copy()
일반_원수_직전3년연간경과보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_직전3년연간경과보험료_가공)
일반_원수_직전3년연간경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_직전3년연간경과보험료_가공)
일반_원수_직전3년연간경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_직전3년연간경과보험료_가공, 일반_상품정보)
일반_원수_직전3년연간경과보험료_집계 = 일반_원수_직전3년연간경과보험료_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['OGL_ELP_PRM']].sum()

일반_출재_직전3년연간경과보험료_가공 = 일반_출재_직전3년연간경과보험료.copy()
일반_출재_직전3년연간경과보험료_가공['RRNR_DVCD'] = '03'
일반_출재_직전3년연간경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_직전3년연간경과보험료_가공)
일반_출재_직전3년연간경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_직전3년연간경과보험료_가공, 일반_상품정보)
일반_출재_직전3년연간경과보험료_집계 = 일반_출재_직전3년연간경과보험료_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['RN_ELP_PRM']].sum()

일반_원수_직전3년연간손해액_가공 = 일반_원수_직전3년연간손해액.copy()
일반_원수_직전3년연간손해액_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_직전3년연간손해액_가공)
일반_원수_직전3년연간손해액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_직전3년연간손해액_가공)
일반_원수_직전3년연간손해액_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_직전3년연간손해액_가공, 일반_상품정보)
일반_원수_직전3년연간손해액_집계 = 일반_원수_직전3년연간손해액_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['OGL_LOSS']].sum()

일반_출재_직전3년연간손해액_가공 = 일반_출재_직전3년연간손해액.copy()
일반_출재_직전3년연간손해액_가공['RRNR_DVCD'] = '03'
일반_출재_직전3년연간손해액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_직전3년연간손해액_가공)
일반_출재_직전3년연간손해액_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_직전3년연간손해액_가공, 일반_상품정보)
일반_출재_직전3년연간손해액_집계 = 일반_출재_직전3년연간손해액_가공.groupby(['FY', 'BOZ_CD'], as_index=False)[['RN_LOSS']].sum()

일반_직전3년평균손해율 = 일반_원수_직전3년연간경과보험료_집계 \
    .merge(일반_출재_직전3년연간경과보험료_집계, on=['FY', 'BOZ_CD'], how='outer') \
    .merge(일반_원수_직전3년연간손해액_집계, on=['FY', 'BOZ_CD'], how='outer') \
    .merge(일반_출재_직전3년연간손해액_집계, on=['FY', 'BOZ_CD'], how='outer')
일반_직전3년평균손해율[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']] = 일반_직전3년평균손해율[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']].fillna(0)
일반_직전3년평균손해율.eval('RET_ELP_PRM = OGL_ELP_PRM - RN_ELP_PRM', inplace=True)
일반_직전3년평균손해율.eval('RET_LOSS = OGL_LOSS - RN_LOSS', inplace=True)
일반_직전3년평균손해율.eval('LOSS_RATIO = RET_LOSS/RET_ELP_PRM', inplace=True)
일반_직전3년평균손해율 = 일반_직전3년평균손해율.groupby('BOZ_CD', as_index=False)['LOSS_RATIO'].mean()

# 특약보종별 직전1년경과보험료
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료.copy()
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료_가공.query('~RRNR_DAT_DVCD.isna()').reset_index(drop=True)
일반_특약보종별_직전1년경과보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_특약보종별_직전1년경과보험료_가공)
일반_특약보종별_직전1년경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_특약보종별_직전1년경과보험료_가공)
일반_특약보종별_직전1년경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_특약보종별_직전1년경과보험료_가공, 일반_상품정보)
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료_가공.merge(일반_특약정보_가공, on='RRNR_TTY_CD', how='left')
일반_특약보종별_직전1년경과보험료_가공 = 일반_특약보종별_직전1년경과보험료_가공.groupby(['TTY_CD_GRP', 'BOZ_CD', 'TTY_YR'], as_index=False)[['OGL_ELP_PRM', 'RN_ELP_PRM']].sum()

# 특약보종별 직전1년손해액
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액.copy()
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액_가공.query('~RRNR_DAT_DVCD.isna()').reset_index(drop=True)
일반_특약보종별_직전1년손해액_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_특약보종별_직전1년손해액_가공)
일반_특약보종별_직전1년손해액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_특약보종별_직전1년손해액_가공)
일반_특약보종별_직전1년손해액_가공['BOZ_CD'] = clsf_boz_cd(일반_특약보종별_직전1년손해액_가공, 일반_상품정보)
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액_가공.merge(일반_특약정보_가공, on='RRNR_TTY_CD', how='left')
일반_특약보종별_직전1년손해액_가공 = 일반_특약보종별_직전1년손해액_가공.groupby(['TTY_CD_GRP', 'BOZ_CD', 'TTY_YR'], as_index=False)[['OGL_LOSS', 'RN_LOSS']].sum()

일반_특약보종별_실적 = 일반_특약보종별_직전1년경과보험료_가공 \
    .merge(일반_특약보종별_직전1년손해액_가공, on=['TTY_CD_GRP', 'BOZ_CD', 'TTY_YR'], how='outer')
일반_특약보종별_실적[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']] = 일반_특약보종별_실적[['OGL_ELP_PRM', 'RN_ELP_PRM', 'OGL_LOSS', 'RN_LOSS']].fillna(0)

# 특약보종별 실적 집계
# 일반_특약보종별_실적_가공 = []
# for tty_cd_grp in ['재물보험특약', '기술보험특약', '근재보험특약', '배상책임보험특약', '해외PST']:
#     STRT_YYYY = str(int(BASE_YYYY)  - (7 if tty_cd_grp in ['근재보험특약', '배상책임보험특약'] else 5))
#     일반_특약보종별_실적_가공.append(일반_특약보종별_실적.query('TTY_CD_GRP == @tty_cd_grp').query('TTY_YR > @STRT_YYYY'))
# 일반_특약보종별_실적_가공 = pd.concat(일반_특약보종별_실적_가공, axis=0).reset_index(drop=True)
일반_특약보종별_실적_가공 = []
for boz_cd in ['A001', 'A002', 'A003', 'A004', 'A005', 'A006', 'A007', 'A008', 'A009', 'A010', 'A011']:
    STRT_YYYY = str(int(BASE_YYYY)  - (7 if boz_cd in ['A005', 'A006'] else 5))
    일반_특약보종별_실적_가공.append(일반_특약보종별_실적.query('BOZ_CD == @boz_cd').query('TTY_YR > @STRT_YYYY'))
일반_특약보종별_실적_가공 = pd.concat(일반_특약보종별_실적_가공, axis=0).reset_index(drop=True)
일반_특약보종별_실적_가공

C:\Users\HOME-Notebook\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,TTY_CD_GRP,BOZ_CD,TTY_YR,OGL_ELP_PRM,RN_ELP_PRM,OGL_LOSS,RN_LOSS
0,재물보험특약,A001,2015,"-9,185,070.000","-2,856,855.000","-21,487,314.000","-8,594,925.000"
1,재물보험특약,A001,2016,"192,314,618.000","38,955,682.000","-294,201,821.000","-58,840,364.000"
2,재물보험특약,A001,2017,"527,012,948.000","162,319,990.000","-648,909.000","-2,236,364.000"
3,재물보험특약,A001,2018,"19,169,845,815.000","3,882,903,831.000","1,886,550,250.000","373,324,085.000"
4,재물보험특약,A001,2019,"18,507,879,363.000","2,880,776,859.000","6,446,951,323.000","1,183,900,278.000"
5,기술보험특약,A002,2015,"37,594,020.000","15,393,040.000","164,251.000","58,900.000"
6,기술보험특약,A002,2016,"207,109,805.000","50,539,226.000","-32,822,816.000","-6,659,421.000"
7,기술보험특약,A002,2017,"718,704,908.000","131,154,554.000","23,313,654.000","-3,466,738.000"
8,기술보험특약,A002,2018,"2,149,270,575.000","541,190,673.000","50,543,216.000","9,682,713.000"
9,기술보험특약,A002,2019,"2,520,270,388.000","817,291,327.000","190,887,270.000","43,677,454.000"


In [7]:

일반_특약보종별_실적

,TTY_CD_GRP,BOZ_CD,TTY_YR,OGL_ELP_PRM,RN_ELP_PRM,OGL_LOSS,RN_LOSS
0,근재보험특약,A005,2013,"159,604.000","28,817.000","52,695,785.000","4,281,157.000"
1,근재보험특약,A005,2014,"16,407,076.000","3,275,285.000","-89,491,526.000","-21,614,499.000"
2,근재보험특약,A005,2015,"14,757,334.000","2,756,334.000","166,145,638.000","33,032,277.000"
3,근재보험특약,A005,2016,"1,134,705,058.000","226,351,667.000","528,395,634.000","92,645,431.000"
4,근재보험특약,A005,2017,"84,867,675.000","15,279,460.000","579,066,615.000","108,302,091.000"
5,근재보험특약,A005,2018,"6,018,237,352.000","1,190,887,076.000","1,190,607,089.000","238,121,416.000"
6,근재보험특약,A005,2019,"8,596,002,347.000","1,700,595,263.000","845,269,690.000","164,253,939.000"
7,기술보험특약,A002,2013,"7,545,087.000","5,607,018.000","350,000.000","315,000.000"
8,기술보험특약,A002,2014,"217,202,262.000","123,040,504.000","3,019,282.000","1,206,864.000"
9,기술보험특약,A002,2015,"37,594,020.000","15,393,040.000","164,251.000","58,900.000"


In [5]:
# 예시
# 근재보험특약, 기술보험특약, 재물보험특약, 배상책임보험특약, 해외PST
특약명 = "해외PST"
prem = 일반_특약보종별_실적_가공.query('TTY_CD_GRP == @특약명').groupby(['TTY_YR', 'BOZ_CD'], as_index=False)[['OGL_ELP_PRM', 'RN_ELP_PRM']].sum()
comm = 일반_특약수수료_가공.query('TTY_CD_GRP == @특약명').drop('TTY_CD_GRP', axis=1)
loss_ratio = 0.6
risk_coef = 0.55
get_ret_risk_rate_by_risk_coef(comm, prem, loss_ratio, risk_coef)

,TTY_YR,BOZ_CD,OGL_ELP_PRM,RN_ELP_PRM,OGL_EXP_LOSS_BASE,OGL_EXP_LOSS_SHOCKED,OGL_EXP_LOSS_DIFF,RET_EXP_LOSS_BASE,RET_EXP_LOSS_SHOCKED,RET_EXP_LOSS_DIFF,COMM_BASE,COMM_SHOCKED,COMM_DIFF
2,2017,A002,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
4,2018,A002,"16,147,124.000","2,422,070.000","9,688,274.400","25,028,042.200","15,339,767.800","8,235,032.400","21,273,833.700","13,038,801.300","725,485.655","532,855.400","-192,630.255"
0,2015,A003,"6,820,828.000","2,486,407.000","4,092,496.800","10,572,283.400","6,479,786.600","2,600,652.600","6,718,352.550","4,117,699.950","745,921.851","497,281.400","-248,640.451"
1,2016,A003,"15,907,341.000","3,717,221.000","9,544,404.600","24,656,378.550","15,111,973.950","7,314,072.000","18,894,686.000","11,580,614.000","1,117,024.910","780,616.410","-336,408.501"
3,2017,A003,"3,369,979,760.000","415,458,240.000","2,021,987,856.000","5,223,468,628.000","3,201,480,772.000","1,772,712,912.000","4,579,508,356.000","2,806,795,444.000","127,753,408.800","91,400,812.800","-36,352,596.000"
5,2018,A003,"40,583,366,917.000","4,826,105,205.000","24,350,020,150.200","62,904,218,721.350","38,554,198,571.150","21,454,357,027.200","55,423,755,653.600","33,969,398,626.400","1,445,569,324.685","1,061,743,145.100","-383,826,179.585"


In [ ]:
def get_ret_risk_rate_by_loss_dist(comm: pd.DataFrame, prem: pd.DataFrame, loss: pd.DataFrame, loss_ratio: float, cv: pd.DataFrame) -> pd.DataFrame:
    """특약별 보유리스크율(손해율분포법)"""